In [17]:

import panhumanpy as ph
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import pooch
from scipy.sparse import csr_matrix
from scipy.io import mmwrite
import re
import anndata
import scanpy.external as sce
import scvi
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import gseapy as gp
from gseapy.plot import dotplot
import math

In [18]:
KPMP_SN_raw=sc.read('/scratch/yongqunh_root/yongqunh0/wruopeng/HuBMAP_KPMP/kpmp-sn-raw-rnaseq.h5ad')
HuBMAP_LK_raw=sc.read('/scratch/yongqunh_root/yongqunh0/wruopeng/HuBMAP_KPMP/LK_raw_updated_2025_june.h5ad')
HuBMAP_RK_raw=sc.read('/scratch/yongqunh_root/yongqunh0/wruopeng/HuBMAP_KPMP/RK_raw_updated_2025_june.h5ad')

In [3]:
print(list(KPMP_SN_raw.obs.columns))
print(list(KPMP_SN_raw.obs['cell_type'].unique()))
print(list(KPMP_SN_raw.obs['tissue'].unique()))
print(list(KPMP_SN_raw.obs['tissue_ontology_term_id'].unique()))

['library_id', 'nCount_RNA', 'nFeature_RNA', 'percent.er', 'percent.mt', 'subclass.l2', 'subclass.l1', 'class', 'experiment_id', 'suspension_type', 'assay_ontology_term_id', 'organism_ontology_term_id', 'donor_id', 'specimen', 'disease_category', 'disease_ontology_term_id', 'eGFR', 'diabetes_history', 'hypertension', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'region', 'percent.cortex', 'percent.medulla', 'tissue_type', 'tissue_ontology_term_id', 'cell_type_ontology_term_id', 'Age_binned', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid']
['kidney interstitial alternatively activated macrophage', 'kidney distal convoluted tubule epithelial cell', 'epithelial cell of proximal tubule', 'T cell', 'kidney loop of Henle thick ascending limb epithelial cell', 'kidney collecting duct intercalated cell', 'kidney interstitial fibrobl

In [4]:
print(list(HuBMAP_RK_raw.obs.columns))
print(list(HuBMAP_RK_raw.obs['predicted_label'].unique()))
print(list(HuBMAP_RK_raw.obs['azimuth_label'].unique()))

['uuid', 'hubmap_id', 'age', 'sex', 'height', 'weight', 'bmi', 'cause_of_death', 'race', 'barcode', 'dataset', 'azimuth_label', 'azimuth_id', 'predicted_CLID', 'predicted_label', 'cl_match_type', 'prediction_score']
['kidney cortex collecting duct principal cell', 'kidney connecting tubule epithelial cell', 'vasa recta descending limb cell', 'kidney inner medulla collecting duct principal cell', 'papillary tips cell', 'renal medullary fibroblast', 'parietal epithelial cell', 'vasa recta ascending limb cell', 'kidney loop of Henle short descending thin limb epithelial cell', 'kidney loop of Henle thin ascending limb epithelial cell', 'kidney interstitial fibroblast', 'kidney resident macrophage', 'kidney loop of Henle long descending thin limb inner medulla epithelial cell', 'kidney loop of Henle medullary thick ascending limb epithelial cell', 'renal interstitial pericyte', 'epithelial cell of proximal tubule segment 3', 'mesangial cell', 'glomerular capillary endothelial cell', 'epith

In [20]:
KPMP_SN_raw.var.columns

Index(['feature_name', 'feature_reference', 'feature_biotype',
       'feature_length'],
      dtype='object')

In [5]:
"ENSG00000272954.1" in KPMP_SN_raw.var["feature_name"].astype(str).values


True

In [19]:
KPMP_shared = {
    "library_id": "dataset_id",
    "tissue_ontology_term_id": "as_id",
    "cell_type_ontology_term_id": "cl_id",
    "cell_type": "cl_label",
    "nCount_RNA": "gene_count",
    "Age_binned": "age",
    "sex": "sex",
    "self_reported_ethnicity": "race",
    "disease": "disease",
}
KPMP_SN_raw.obs['consortium'] = 'KPMP'
KPMP_SN_raw.obs['collection'] = 'KPMP SC RNAseq'

def filter_to_dataset_cell_types(
    adata,
    dataset_names,
    source_col='cell_type'
):
    # count before
    before = adata.n_obs

    # filter mask
    mask = adata.obs[source_col].isin(dataset_names)
    adata_filtered = adata[mask, :].copy()

    # report
    after = adata_filtered.n_obs
    print(
        f"Filtered to dataset cell types ({source_col}): "
        f"kept {after}/{before} cells ({before - after} removed)."
    )

    return adata_filtered

mapping = {
    # exact matches
    'B cell': 'B cell',
    'T cell': 'T cell',
    'conventional dendritic cell': 'conventional dendritic cell',
    'mast cell': 'mast cell',
    'mature NK T cell': 'mature NK T cell',
    'mononuclear phagocyte': 'mononuclear phagocyte',
    'neutrophil': 'neutrophil',
    'non-classical monocyte': 'non-classical monocyte',
    'papillary tips cell': 'papillary tips cell',
    'parietal epithelial cell': 'parietal epithelial cell',
    'plasma cell': 'plasma cell',
    'plasmacytoid dendritic cell, human': 'plasmacytoid dendritic cell, human',
    'podocyte': 'podocyte',
    'kidney interstitial fibroblast': 'kidney interstitial fibroblast',
    'renal interstitial pericyte': 'renal interstitial pericyte',
    # proximal tubule
    'epithelial cell of proximal tubule segment 1': 'epithelial cell of proximal tubule',
    'epithelial cell of proximal tubule segment 2': 'epithelial cell of proximal tubule',
    'epithelial cell of proximal tubule segment 3': 'epithelial cell of proximal tubule',
    # distal convoluted tubule
    'epithelial cell of early distal convoluted tubule': 'kidney distal convoluted tubule epithelial cell',
    'epithelial cell of late distal convoluted tubule': 'kidney distal convoluted tubule epithelial cell',
    # connecting tubule & collecting duct
    'kidney connecting tubule alpha-intercalated cell': 'kidney collecting duct intercalated cell',
    'kidney connecting tubule principal cell': 'kidney collecting duct principal cell',
    'kidney cortex collecting duct intercalated cell': 'kidney collecting duct intercalated cell',
    'kidney outer medulla collecting duct intercalated cell': 'kidney collecting duct intercalated cell',
    'kidney cortex collecting duct principal cell': 'kidney collecting duct principal cell',
    'kidney inner medulla collecting duct principal cell': 'kidney collecting duct principal cell',
    'kidney outer medulla collecting duct principal cell': 'kidney collecting duct principal cell',
    'kidney collecting duct intercalated cell': 'kidney collecting duct intercalated cell',
    'kidney collecting duct principal cell': 'kidney collecting duct principal cell',
    'kidney connecting tubule epithelial cell': 'kidney connecting tubule epithelial cell',
    # loop of Henle
    'kidney loop of Henle cortical thick ascending limb epithelial cell': 
        'kidney loop of Henle thick ascending limb epithelial cell',
    'kidney loop of Henle medullary thick ascending limb epithelial cell': 
        'kidney loop of Henle thick ascending limb epithelial cell',
    'kidney loop of Henle thick ascending limb epithelial cell': 
        'kidney loop of Henle thick ascending limb epithelial cell',
    'kidney loop of Henle thin ascending limb epithelial cell': 
        'kidney loop of Henle thin ascending limb epithelial cell',
    'kidney loop of Henle short descending thin limb epithelial cell': 
        'kidney loop of Henle thin descending limb epithelial cell',
    'kidney loop of Henle long descending thin limb inner medulla epithelial cell': 
        'kidney loop of Henle thin descending limb epithelial cell',
    'kidney loop of Henle long descending thin limb outer medulla epithelial cell': 
        'kidney loop of Henle thin descending limb epithelial cell',
    # endothelial
    'endothelial cell': 'endothelial cell',
}
HuBMAP_LK_raw = filter_to_dataset_cell_types(HuBMAP_LK_raw, mapping, source_col='predicted_label')


HuBMAP_RK_raw = filter_to_dataset_cell_types(HuBMAP_RK_raw, mapping, source_col='predicted_label')



def remove_slide_seq_cells(adata, slide_seq_ids):

    # Count before filtering
    total_before = adata.n_obs
    
    # Mask of cells to remove
    remove_mask = adata.obs['hubmap_id'].isin(slide_seq_ids)
    
    # Subset to keep only non-slide_seq cells
    adata_filtered = adata[~remove_mask, :].copy()
    total_after = adata_filtered.n_obs
    
    # Report
    print("Removing slide_seq cells only:")
    print(f"  Cells before: {total_before}")
    print(f"  Cells after:  {total_after}")
    print(f"  Cells removed: {total_before - total_after}")
    return adata_filtered
need_to_removed_slide_seq_HuBMAP=['HBM222.VQSW.335',
 'HBM248.HPXX.584',
 'HBM269.GDLH.894',
 'HBM294.XZLM.256',
 'HBM356.MDPN.792',
 'HBM363.FVKP.935',
 'HBM398.BLRW.228',
 'HBM456.CGDP.395',
 'HBM462.XQCR.933',
 'HBM522.QXVG.468',
 'HBM528.KNCB.488',
 'HBM547.SJSK.268',
 'HBM547.TFRR.794',
 'HBM595.LBXP.486',
 'HBM679.RLJH.432',
 'HBM735.FSBZ.626',
 'HBM757.KLKW.524',
 'HBM775.CMGG.464',
 'HBM785.XFTT.663',
 'HBM834.SLQN.292',
 'HBM883.PHQS.523',
 'HBM976.*','HBM232.MBNR.586', 'HBM266.FTJN.632', 'HBM297.FDTX.382', 'HBM445.HBRO.488',
 'HBM459.KCST.593', 'HBM532.KKRC.477', 'HBM634.JHVB.286', 'HBM634.ZSHF.736',
 'HBM647.QDBG.936', 'HBM736.MNMD.453', 'HBM827.MJMM.447', 'HBM846.KVCF.674',
 'HBM892.CCDZ.345', 'HBM965.PSNC.855', 'HBM986.KFWG.239', 'HBM232.MBNR.586']
HuBMAP_LK_raw=remove_slide_seq_cells(HuBMAP_LK_raw,need_to_removed_slide_seq_HuBMAP)
HuBMAP_RK_raw=remove_slide_seq_cells(HuBMAP_RK_raw,need_to_removed_slide_seq_HuBMAP)


KPMP_SN_raw.obs.rename(columns=KPMP_shared, inplace=True)

KPMP_var_shared = {
    'feature_name': 'gene_name'
}
KPMP_SN_raw.var.rename(columns=KPMP_var_shared, inplace=True)

Hubmap_shared = {
    "cell_id": "cell_id", 
    "predicted_CLID": "cl_id",
    "predicted_label": "cl_label",
    "n_genes": "gene_count",
    "age": "age",
    "sex": "sex",
    "race": "race",
    "hubmap_id":"dataset_id"
}
HubMap_var_shared = {
    "hugo_symbol": "gene_name"
}

def add_common_value(adata,RK):
 
    adata.obs['as_id'] = "UBERON:0002113"
    adata.obs['disease'] = "normal"
    adata.obs['consortium'] = 'HuBMAP'
    if RK == True:
        adata.obs['collection'] = 'HuBMAP Right Kidney'
    else:
        adata.obs['collection'] = 'HuBMAP Left Kidney'
    adata.obs['tissue'] = 'kidney'
    # Rename HuBMAP columns using inplace renaming
    adata.obs.rename(columns=Hubmap_shared, inplace=True)
    adata.var.rename(columns=HubMap_var_shared, inplace=True)
    adata.var.index = adata.var.index.to_series().apply(lambda x: re.sub(r'\.\d+$', '', x))
    return adata

HuBMAP_LK_raw = add_common_value(HuBMAP_LK_raw,False)
HuBMAP_RK_raw = add_common_value(HuBMAP_RK_raw,True)


def  remove_extra_var(adata):
    columns_to_drop = [col for col in adata.var.columns if col != 'gene_name']
    adata.var.drop(columns=columns_to_drop, inplace=True)
    return adata

KPMP_SN_raw=remove_extra_var(KPMP_SN_raw)

def drop_unqiue_obs_columns(adata):
    keep_cols = [
    'consortium',
    'collection',
    'dataset_id',
    'as_id',
    'cl_id',
    'cl_label',
    # 'gene_count',
    'age',
    'sex',
    'race',
    'disease'
    ]
    missing_cols = [col for col in keep_cols if col not in adata.obs.columns]
    if missing_cols:
        print("Warning: The following columns are missing in adata.obs:", missing_cols)
    available_cols = [col for col in keep_cols if col in adata.obs.columns]
    adata.obs = adata.obs[available_cols]
    return adata

HuBMAP_LK_raw = drop_unqiue_obs_columns(HuBMAP_LK_raw)
HuBMAP_RK_raw = drop_unqiue_obs_columns(HuBMAP_RK_raw)

# HuBMAP_combined=drop_unqiue_obs_columns(HuBMAP_combined)

KPMP_SN_raw=drop_unqiue_obs_columns(KPMP_SN_raw)

def map_the_gene_name(adata):
    common_genes = KPMP_SN_raw.var.index.intersection(adata.var.index)
    
    gene_names = KPMP_SN_raw.var.loc[common_genes, 'gene_name'].astype(str)
    
    adata.var['gene_name'] = adata.var.get('gene_name', pd.Series(dtype=str)).astype(str)
    
    adata.var.loc[common_genes, 'gene_name'] = gene_names
    
    print(f"Identified and updated {len(common_genes)} common gene IDs.")
    
    
    return adata

HuBMAP_LK_raw = map_the_gene_name(HuBMAP_LK_raw)
HuBMAP_RK_raw = map_the_gene_name(HuBMAP_RK_raw)


shared = (
    KPMP_SN_raw.var.index
    .intersection(HuBMAP_LK_raw.var.index)
    .intersection(HuBMAP_RK_raw.var.index)
)

print(f"Found {len(shared)} genes shared across all datasets.")

KPMP_SN_raw     = KPMP_SN_raw[:, shared].copy()
HuBMAP_LK_raw   = HuBMAP_LK_raw[:, shared].copy()
HuBMAP_RK_raw   = HuBMAP_RK_raw[:, shared].copy()
print("Obs columns before cocnat:", KPMP_SN_raw.obs.columns.tolist())

print("Obs columns before cocnat:", HuBMAP_LK_raw.obs.columns.tolist())

print("Obs columns before cocnat:", HuBMAP_RK_raw.obs.columns.tolist())
adata_concat_scvi = anndata.concat([KPMP_SN_raw,HuBMAP_LK_raw,HuBMAP_RK_raw], label="batch", keys=["KPMP", "HuBMAP_LK", "HuBMAP_RK"],axis=0, join="inner", merge="first")



Filtered to dataset cell types (predicted_label): kept 1371041/1522897 cells (151856 removed).
Filtered to dataset cell types (predicted_label): kept 1673578/1803736 cells (130158 removed).
Removing slide_seq cells only:
  Cells before: 1371041
  Cells after:  960932
  Cells removed: 410109
Removing slide_seq cells only:
  Cells before: 1673578
  Cells after:  1283320
  Cells removed: 390258
Identified and updated 36297 common gene IDs.
Identified and updated 36297 common gene IDs.
Found 36297 genes shared across all datasets.
Obs columns before cocnat: ['consortium', 'collection', 'dataset_id', 'as_id', 'cl_id', 'cl_label', 'age', 'sex', 'race', 'disease']
Obs columns before cocnat: ['consortium', 'collection', 'dataset_id', 'as_id', 'cl_id', 'cl_label', 'age', 'sex', 'race', 'disease']
Obs columns before cocnat: ['consortium', 'collection', 'dataset_id', 'as_id', 'cl_id', 'cl_label', 'age', 'sex', 'race', 'disease']


In [20]:
adata_concat_scvi_1=adata_concat_scvi.copy()
sc.pp.filter_cells(adata_concat_scvi, min_genes=200) 
sc.pp.filter_genes (adata_concat_scvi, min_cells=3) 


In [8]:
print(adata_concat_scvi.X)

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 1009461054 stored elements and shape (832343, 35878)>
  Coords	Values
  (0, 44)	1.0
  (0, 74)	1.0
  (0, 678)	1.0
  (0, 1638)	1.0
  (0, 1949)	1.0
  (0, 2157)	1.0
  (0, 3179)	1.0
  (0, 3226)	1.0
  (0, 3594)	1.0
  (0, 4418)	1.0
  (0, 4873)	1.0
  (0, 5484)	1.0
  (0, 6447)	1.0
  (0, 6495)	1.0
  (0, 6581)	5.0
  (0, 6612)	1.0
  (0, 7172)	1.0
  (0, 7236)	1.0
  (0, 7714)	1.0
  (0, 7716)	1.0
  (0, 8081)	1.0
  (0, 8113)	1.0
  (0, 8120)	1.0
  (0, 8170)	3.0
  (0, 8206)	1.0
  :	:
  (832342, 35631)	1.0
  (832342, 35641)	0.5
  (832342, 35670)	1.0
  (832342, 35681)	0.25
  (832342, 35682)	1.0
  (832342, 35696)	0.33333298563957214
  (832342, 35714)	0.5
  (832342, 35741)	1.0
  (832342, 35755)	0.5
  (832342, 35782)	1.0
  (832342, 35785)	1.0
  (832342, 35786)	0.5
  (832342, 35796)	0.5
  (832342, 35808)	0.5
  (832342, 35809)	0.5
  (832342, 35811)	1.0
  (832342, 35812)	1.0
  (832342, 35813)	1.0
  (832342, 35830)	0.5
  (832342, 35835)	0.5
  (832342,

In [9]:
sc.pp.normalize_total(adata_concat_scvi,target_sum=1e4)
# sc.pp.log1p(adata_concat_scvi)

In [22]:
label_mapping = {
    # Proximal tubule
    "epithelial cell of proximal tubule": "Proximal tubule epithelial cell",
    "epithelial cell of proximal tubule segment 1": "Proximal tubule epithelial cell",
    "epithelial cell of proximal tubule segment 2": "Proximal tubule epithelial cell",
    "epithelial cell of proximal tubule segment 3": "Proximal tubule epithelial cell",

    # Distal convoluted tubule
    "kidney distal convoluted tubule epithelial cell": "Distal convoluted tubule epithelial cell",
    "epithelial cell of early distal convoluted tubule": "Distal convoluted tubule epithelial cell",
    "epithelial cell of late distal convoluted tubule": "Distal convoluted tubule epithelial cell",

    # Loop of Henle
    "kidney loop of Henle thin descending limb epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle thin ascending limb epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle cortical thick ascending limb epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle medullary thick ascending limb epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle long descending thin limb outer medulla epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle long descending thin limb inner medulla epithelial cell": "Loop of Henle epithelial cell ",
    "kidney loop of Henle short descending thin limb epithelial cell": "Loop of Henle epithelial cell ",

    # Collecting duct principal cells
    "kidney collecting duct principal cell": "Collecting duct principal cell",
    "kidney cortex collecting duct principal cell": "Collecting duct principal cell",
    "kidney outer medulla collecting duct principal cell": "Collecting duct principal cell",
    "kidney inner medulla collecting duct principal cell": "Collecting duct principal cell",

    # Collecting duct intercalated cells
    "kidney collecting duct intercalated cell": "Collecting duct intercalated cell",
    "kidney cortex collecting duct intercalated cell": "Collecting duct intercalated cell",
    "kidney outer medulla collecting duct intercalated cell": "Collecting duct intercalated cell",

    # Connecting tubule
    "kidney connecting tubule epithelial cell": "Connecting tubule epithelial cell",
    "kidney connecting tubule principal cell": "Connecting tubule epithelial cell",
    "kidney connecting tubule alpha-intercalated cell": "Connecting tubule epithelial cell",

    # Other epithelial
    "podocyte": "Podocyte",
    "parietal epithelial cell": "Parietal epithelial cell",
    "papillary tips cell": "Papillary tips cell",

    # Stromal & endothelial
    "endothelial cell": "Endothelial cell",
    "kidney interstitial cell": "Kidney interstitial cell",
    "kidney interstitial fibroblast": "Kidney interstitial fibroblast",
    "kidney interstitial alternatively activated macrophage": "Kidney interstitial alternatively activated macrophage",
    "renal interstitial pericyte": "Renal interstitial pericyte",

    # Immune cells
    "T cell": "T cell",
    "cytotoxic T cell": "T cell",
    "mature NK T cell": "T cell",
    "B cell": "B cell",
    "plasma cell": "Plasma cell",
    "natural killer cell": "Natural killer cell",
    "monocyte": "Monocyte",
    "non-classical monocyte": "Monocyte",
    "conventional dendritic cell": "Dendritic cell",
    "plasmacytoid dendritic cell, human": "Dendritic cell",
    "mononuclear phagocyte": "Mononuclear phagocyte",
    "mast cell": "Mast cell",
    "neutrophil": "Neutrophil",
}

adata_concat_scvi.obs["general_cl_label"] = adata_concat_scvi.obs["cl_label"].map(label_mapping)

adata_concat_scvi.obs["general_cl_label"] = adata_concat_scvi.obs["general_cl_label"].fillna(adata_concat_scvi.obs["cl_label"])
adata_concat_scvi.obs['age'] = adata_concat_scvi.obs['age'].astype(str)

print(adata_concat_scvi.obs[["cl_label", "general_cl_label"]].head(20))

                                                                  cl_label  \
KC33_AAACCCAAGTAGTCAA-1  kidney interstitial alternatively activated ma...   
KC33_AAACCCACAACTGAAA-1    kidney distal convoluted tubule epithelial cell   
KC33_AAACCCACACGTTGGC-1                 epithelial cell of proximal tubule   
KC33_AAACCCAGTACATACC-1                                             T cell   
KC33_AAACCCATCCGAGCTG-1                 epithelial cell of proximal tubule   
KC33_AAACCCATCTTCGTGC-1                 epithelial cell of proximal tubule   
KC33_AAACGAACAACGATCT-1  kidney loop of Henle thick ascending limb epit...   
KC33_AAACGAACACCGTGGT-1  kidney loop of Henle thick ascending limb epit...   
KC33_AAACGAACAGTAGAAT-1                                             T cell   
KC33_AAACGAACAGTATACC-1                 epithelial cell of proximal tubule   
KC33_AAACGAAGTAGGCAGT-1                 epithelial cell of proximal tubule   
KC33_AAACGAAGTGCATGAG-1                 epithelial cell of proxi

In [28]:
adata_concat_scvi.obs["general_cl_label"].unique()

array(['Kidney interstitial alternatively activated macrophage',
       'Distal convoluted tubule epithelial cell',
       'Proximal tubule epithelial cell', 'T cell',
       'kidney loop of Henle thick ascending limb epithelial cell',
       'Collecting duct intercalated cell',
       'Kidney interstitial fibroblast',
       'Connecting tubule epithelial cell',
       'Collecting duct principal cell', 'Endothelial cell', 'B cell',
       'Mononuclear phagocyte', 'Podocyte', 'Parietal epithelial cell',
       'Mast cell', 'Dendritic cell', 'Plasma cell', 'Monocyte',
       'Loop of Henle epithelial cell ', 'Renal interstitial pericyte',
       'Neutrophil', 'Papillary tips cell', 'neural cell'], dtype=object)

In [7]:
print(f"Total Cells: {adata_concat_scvi.n_obs:,}")
print(f"Total Genes: {adata_concat_scvi.n_vars:,}")  
adata_concat_scvi.write_h5ad("/scratch/yongqunh_root/yongqunh0/wruopeng/HuBMAP_KPMP/kpmp_hubmap_combined_after_1e4_normalization_only_afterQC.h5ad")


Total Cells: 832,343
Total Genes: 35,878


In [11]:
print(f"Total Cells: {adata_concat_scvi.n_obs:,}")
print(f"Total Genes: {adata_concat_scvi.n_vars:,}")  

Total Cells: 832,343
Total Genes: 35,878


In [23]:
adata_concat_scvi.var_names=adata_concat_scvi.var['gene_name']

/home/wruopeng/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:845: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']

    Inferred to be: categorical

  names = self._prep_dim_index(names, "var")


In [ ]:
import pandas as pd

csv1 = "uberonID.csv"
csv2 = "cl_mapping_edge.csv"
csv3 = "genes_nodes.csv"

def load_and_fix(path):
    df = pd.read_csv(path)
    df.columns = [c.strip().lower() for c in df.columns]
    df = df.rename(columns={"types": "type"})
    return df

df1 = load_and_fix(csv1)
df2 = load_and_fix(csv2)
df3 = load_and_fix(csv3)

common_cols = set(df1.columns) & set(df2.columns) & set(df3.columns)
df1 = df1[list(common_cols)]
df2 = df2[list(common_cols)]
df3 = df3[list(common_cols)]

combined = pd.concat([df1, df2, df3], ignore_index=True).drop_duplicates()

# Save
combined.to_csv("combined_nodes.csv", index=False)

print("Saved as combined_clean.csv")


Saved as combined_clean.csv


In [ ]:
import re
import requests
import pandas as pd

FIXED_TYPE   = "http://purl.obolibrary.org/obo/UBERON_0000061"
FIXED_SOURCE = "https://purl.humanatlas.io/vocab/hp"

_SUFFIXES = [
    r"\bepithelial cells?\b", r"\bepithelial cell\b",
    r"\bprincipal cells?\b",  r"\bintercalated cells?\b",
    r"\bfibroblasts?\b",      r"\bpericytes?\b",
    r"\bendothelial cells?\b",r"\bmesangial cells?\b",
    r"\bpodocytes?\b",        r"\bimmune cells?\b",
    r"\b(neutrophils?|monocytes?|macrophages?|dendritic cells?)\b",
    r"\b(T|B) cells?\b",      r"\bstem cells?\b",
    r"\bplasma cells?\b",     r"\bmast cells?\b",
    r"\bneurons?\b",          r"\bglial cells?\b",
    r"\binterstitial cells?\b", r"\bmyofibroblasts?\b",
    r"\bprincipal cell\b",    r"\bintercalated cell\b",
    r"\bcell(?:s)?\b"
]
_SUFFIX_RE = re.compile("|".join(_SUFFIXES), flags=re.IGNORECASE)

def _clean_term_for_uberon(term: str) -> str:
    cleaned = _SUFFIX_RE.sub("", term)
    cleaned = re.sub(r"\s+", " ", cleaned).strip(" ,;.-").strip()
    return cleaned or term  # never return empty

def _search_ols_uberon(q: str):
    """Call OLS search and return docs list (or [])."""
    url = "https://www.ebi.ac.uk/ols/api/search"
    params = {
        "q": q,
        "ontology": "uberon",  
        "type": "class",       
        "exact": "false",
        "rows": 50
    }
    try:
        r = requests.get(url, params=params, timeout=20)
        if r.status_code == 200:
            j = r.json()
            return j.get("response", {}).get("docs", []) or []
    except Exception:
        pass
    return []

def _pick_uberon_hit(docs):
    """Return the first doc that is definitely UBERON."""
    for d in docs:
        iri = d.get("iri", "") or ""
        obo = d.get("obo_id", "") or ""
        onto = (d.get("ontology_name") or "").lower()
        if onto == "uberon" or "UBERON_" in iri or obo.startswith("UBERON:"):
            return d
    return None

def get_uberon_entry_strict(term: str):
    """
    Return dict with iri, label, type, source.
    Try original term, then a 'cleaned' term.
    If no UBERON hit is found, return kidney default.
    """
    term = "" if term is None else str(term).strip()
    # 1) try raw
    docs = _search_ols_uberon(term)
    hit = _pick_uberon_hit(docs)

    if hit is None:
        cleaned = _clean_term_for_uberon(term)
        if cleaned and cleaned.lower() != term.lower():
            docs2 = _search_ols_uberon(cleaned)
            hit = _pick_uberon_hit(docs2)

    if hit:
        return {
            "iri": hit.get("iri"),
            "label": hit.get("label"),
            "type": FIXED_TYPE,
            "source": FIXED_SOURCE,
        }

    return {
        "iri": "http://purl.obolibrary.org/obo/UBERON_0002113",
        "label": "kidney",
        "type": FIXED_TYPE,
        "source": FIXED_SOURCE,
    }

terms = pd.unique(adata_concat_scvi.obs['general_cl_label']).tolist()

rows = [get_uberon_entry_strict(t) for t in terms]
df = pd.DataFrame(rows, columns=["iri", "label", "type", "source"])
df.to_csv("uberon_mapping_edge1.csv", index=False)
print(f"✅ Saved uberon_mapping.csv with {len(df)} rows (UBERON only + kidney fallback).")



✅ Saved uberon_mapping.csv with 23 rows (UBERON only + kidney fallback).


In [ ]:
import requests
import pandas as pd

def get_hgnc_iri(gene_symbol: str) -> str:
    """
    Query Identifiers.org / HGNC to resolve a gene symbol to its iri.
    Falls back to empty string if not found.
    """
    url = f"https://rest.genenames.org/fetch/symbol/{gene_symbol}"
    headers = {"Accept": "application/json"}
    
    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()
        docs = r.json().get("response", {}).get("docs", [])
        if docs:
            hgnc_id = docs[0].get("hgnc_id", "")
            if hgnc_id:
                # hgnc_id is like "HGNC:132", so we split and take the number
                number = hgnc_id.split(":")[1]
                return f"http://identifiers.org/hgnc/{number}"
    except Exception as e:
        print(f"Failed to fetch {gene_symbol}: {e}")
    return ""

# gene_names = adata_concat_scvi.var['gene_name'].tolist()
gene_names = [        "CCL21",
    "NRXN1",
    "SLC9A3",
    "SPP1",
    "SERPINE2",
    "IL18",
    "NBAS",
    "TIMP2",
    "IGFBP7",
    "SUGP1",
    "APOA1",
    "MMP9",
    "ITGAM",
    "TFF3",
    "B2M",
    "S100A6",
    "IL10",
    "IL6",
    "CXCL8",
    "LCN2",
    "HAVCR1",
    "CST3",
    "CCL2",
    "FABP1"]


iris = [get_hgnc_iri(g) for g in gene_names]

df = pd.DataFrame({
    "iri": iris,
    "label": gene_names,
    "type": ["http://purl.bioontology.org/ontology/HGNC/gene"] * len(gene_names),
    "source": ["https://purl.humanatlas.io/vocab/hp"] * len(gene_names)
})

df.to_csv("genes_nodes.csv", index=False)
print("genes.csv created with real HGNC iris.")


genes.csv created with real HGNC iris.


In [ ]:

def get_cl_info(term):
    """Query OLS for a Cell Ontology (CL) term given a cell type name."""
    url = "https://www.ebi.ac.uk/ols/api/search"
    params = {"q": term, "ontology": "cl", "exact": "false"}
    r = requests.get(url, params=params)

    if r.status_code == 200:
        data = r.json()
        if data['response']['numFound'] > 0:
            hit = data['response']['docs'][0]
            return {
                "iri": hit.get("iri"),
                "label": hit.get("label"),
                "types": "http://purl.obolibrary.org/obo/CL_0000000",  # e.g. CL_0000236
                "source": "https://www.ebi.ac.uk/ols/ontologies/cl"
            }
    return {
        "iri": "NA",
        "label": "NA",
        "types": "http://purl.obolibrary.org/obo/CL_0000000",
        "source": "https://www.ebi.ac.uk/ols/ontologies/cl"
    }

terms = adata_concat_scvi.obs['general_cl_label'].unique().tolist()

results = [get_cl_info(t) for t in terms]

df = pd.DataFrame(results)

df.to_csv("cl_mapping_edge.csv", index=False)

print("✅ Saved cl_mapping.csv with", len(df), "unique cell type mappings")


✅ Saved cl_mapping.csv with 23 unique cell type mappings


In [11]:
adata_fem = adata_concat_scvi[adata_concat_scvi.obs["sex"] == "male"].copy()

adata_aki = adata_fem[adata_fem.obs["disease"] == "acute kidney failure"].copy()
adata_ckd = adata_fem[adata_fem.obs["disease"] == "chronic kidney disease"].copy()
adata_normal = adata_fem[adata_fem.obs["disease"] == "normal"].copy()

# Print sizes
print("AKI subset:", adata_aki.shape)      
print("CKD subset:", adata_ckd.shape)
print("Normal subset:", adata_normal.shape)

AKI subset: (41512, 35878)
CKD subset: (62983, 35878)
Normal subset: (58754, 35878)


In [9]:
print(adata_concat_scvi.obs['dataset_id'].unique())

['KC33', 'KC34', 'KC35', 'KC36', 'PA1', ..., 'HBM445.HBRQ.488', 'HBM729.TVMN.534', 'HBM478.VWXX.362', 'HBM847.MDSJ.826', 'HBM766.NZWP.682']
Length: 181
Categories (181, object): ['HBM226.HRTZ.874', 'HBM247.HLXR.494', 'HBM253.ZBGF.863', 'HBM264.MJCH.639', ..., 'PA1', 'PA3', 'PA4', 'PA5']


In [10]:
marker_genes=[        "CCL21",
    "NRXN1",
    "SLC9A3",
    "SPP1",
    "SERPINE2",
    "IL18",
    "NBAS",
    "TIMP2",
    "IGFBP7",
    "SUGP1",
    "APOA1",
    "MMP9",
    "ITGAM",
    "TFF3",
    "B2M",
    "S100A6",
    "IL10",
    "IL6",
    "CXCL8",
    "LCN2",
    "HAVCR1",
    "CST3",
    "CCL2",
    "FABP1"]

In [46]:
sc.tl.rank_genes_groups(
    adata_concat_scvi,
    groupby="disease",        
    method="t-test",          
    reference="normal"           
)

/home/wruopeng/.local/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:459: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/home/wruopeng/.local/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:461: RuntimeWarning: divide by zero encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/home/wruopeng/.local/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:459: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/home/wruopeng/.local/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:461: RuntimeWarning: divide by zero encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(


In [88]:
header=['biomarker_id','biomarker_label','cell_types','disease_types','mean_expression_value","fold changes"]
df_KPMP_HuBMAP_integrated = pd.DataFrame(columns=header)

SyntaxError: unterminated string literal (detected at line 1) (951358321.py, line 1)

In [23]:
print(adata_concat_scvi.obs['disease'].unique())

['normal', 'chronic kidney disease', 'acute kidney failure']
Categories (3, object): ['acute kidney failure', 'chronic kidney disease', 'normal']


In [ ]:


import requests
import pandas as pd

def get_uberon_info(term):
    """Query OLS for a UBERON term given a tissue/structure name."""
    url = "https://www.ebi.ac.uk/ols/api/search"
    params = {"q": term, "ontology": "uberon", "exact": "false"}
    r = requests.get(url, params=params)

    if r.status_code == 200:
        data = r.json()
        if data['response']['numFound'] > 0:
            hit = data['response']['docs'][0]
            return {
                "iri": hit.get("iri"),
                "label": hit.get("label"),
                "type": "http://purl.obolibrary.org/obo/UBERON_0000061",
                "source": "https://purl.humanatlas.io/vocab/hp"
            }
    return {
        "iri": "http://purl.obolibrary.org/obo/UBERON_0002113",
        "label": "kidney",
        "type": "UBERON:0002113",
        "source": "https://purl.humanatlas.io/vocab/hp"
    }

def map_celltypes_to_uberon(df_in):
    results = []
    for cl_label, tissue in zip(df_in["cl_label"], df_in["tissue"]):
        
        info = get_uberon_info(tissue)
        if any(x in cl_label.lower() for x in [
            "proximal", "distal", "loop of henle", "collecting", 
            "podocyte", "parietal", "fibroblast", "pericyte", "papillary", "interstitium"
        ]):
            label_val = "kidney " + tissue.lower()
        else:
            label_val = "kidney"

        info["cell_type"] = cl_label
        info["tissue"] = tissue
        info["label"] = label_val  
        results.append(info)
    return pd.DataFrame(results)

celltypes = list(adata_concat_scvi.obs['general_cl_label'].unique())
df_input = pd.DataFrame({
    "cl_label": celltypes,
    "tissue": [name.replace("cell", "").replace("Cell", "").strip() for name in celltypes]
})

df_UBERON = map_celltypes_to_uberon(df_input)

df_UBERON.to_csv("celltypes_uberon.csv", index=False, quoting=1)
print(df_UBERON.head())





                                             iri  \
0  http://purl.obolibrary.org/obo/UBERON_0005215   
1  http://purl.obolibrary.org/obo/UBERON_0001292   
2  http://purl.obolibrary.org/obo/UBERON_0004134   
3  http://purl.obolibrary.org/obo/UBERON_0010393   
4  http://purl.obolibrary.org/obo/UBERON_0001291   

                                               label  \
0                                             kidney   
1         kidney distal convoluted tubule epithelial   
2                  kidney proximal tubule epithelial   
3                                             kidney   
4  kidney kidney loop of henle thick ascending li...   

                                            type  \
0  http://purl.obolibrary.org/obo/UBERON_0000061   
1  http://purl.obolibrary.org/obo/UBERON_0000061   
2  http://purl.obolibrary.org/obo/UBERON_0000061   
3  http://purl.obolibrary.org/obo/UBERON_0000061   
4  http://purl.obolibrary.org/obo/UBERON_0000061   

                                sourc

In [ ]:
import pandas as pd
import numpy as np

def mean_expression_compare_to_normal(adata, celltype_key, disease_key,
                                      disease_list, selected_celltypes=None, selected_genes=None,
                                      save_path=None):
    results = []

    for disease in disease_list:
        disease_mask = (adata.obs[disease_key] == disease)
        if selected_celltypes is not None:
            disease_mask &= adata.obs[celltype_key].isin(selected_celltypes)
        adata_dis = adata[disease_mask, :]

        normal_mask = (adata.obs[disease_key] == "normal")
        if selected_celltypes is not None:
            normal_mask &= adata.obs[celltype_key].isin(selected_celltypes)
        adata_norm = adata[normal_mask, :]

        if selected_genes is not None:
            adata_dis = adata_dis[:, selected_genes]
            adata_norm = adata_norm[:, selected_genes]

        for celltype in adata_dis.obs[celltype_key].unique():
            idx_dis = np.where(adata_dis.obs[celltype_key] == celltype)[0]
            idx_norm = np.where(adata_norm.obs[celltype_key] == celltype)[0]

            if len(idx_dis) == 0 or len(idx_norm) == 0:
                continue

            X_dis = adata_dis[idx_dis, :].X
            X_norm = adata_norm[idx_norm, :].X

            mean_dis = np.array(X_dis.mean(axis=0)).ravel()
            mean_norm = np.array(X_norm.mean(axis=0)).ravel()
            
            dis_sex_counts = adata_dis.obs.iloc[idx_dis]["sex"].value_counts().to_dict()
            norm_sex_counts = adata_norm.obs.iloc[idx_norm]["sex"].value_counts().to_dict()

            num_female_dis = dis_sex_counts.get("female", 0)
            num_male_dis = dis_sex_counts.get("male", 0)
            num_female_norm = norm_sex_counts.get("female", 0)
            num_male_norm = norm_sex_counts.get("male", 0)
            
            dis_datasets = adata_dis.obs.iloc[idx_dis]['dataset_id'].unique().tolist()
            norm_datasets = adata_norm.obs.iloc[idx_norm]['dataset_id'].unique().tolist()

            num_dis_datasets = len(dis_datasets)
            num_norm_datasets = len(norm_datasets)

            for gene, val_dis, val_norm in zip(adata_dis.var_names, mean_dis, mean_norm):
                results.append({
                    "gene": gene,
                    "disease": disease,
                    "cell_type": celltype,
                    "as_id": as_id,
                    "mean_expression_disease": val_dis,
                    "mean_expression_normal": val_norm,
                    "disease_sample_size": len(idx_dis),
                    "normal_sample_size": len(idx_norm),
                    "disease_female_sample_size": num_female_dis,
                    "disease_male_sample_size": num_male_dis,
                    "normal_female_sample_size": num_female_norm,
                    "normal_male_sample_size": num_male_norm,
                    "disease_num_datasets": num_dis_datasets,
                    "normal_num_datasets": num_norm_datasets,
                    "disease_datasets": ",".join(dis_datasets),
                    "normal_datasets": ",".join(norm_datasets),
                })


    tidy_df = pd.DataFrame(results)

    if save_path:
        tidy_df.to_csv(save_path, index=False)
        print(f"✅ Saved to {save_path}")

    return tidy_df



In [99]:
print(adata_concat_scvi.obs.columns)

Index(['consortium', 'collection', 'dataset_id', 'as_id', 'cl_id', 'cl_label',
       'age', 'sex', 'race', 'disease', 'batch', 'n_genes',
       'general_cl_label'],
      dtype='object')


In [147]:
print(adata_concat_scvi.obs['general_cl_label'].unique())
print(len(adata_concat_scvi.obs['general_cl_label'].unique()))

['Kidney interstitial alternatively activated macrophage'
 'Distal convoluted tubule epithelial cell'
 'Proximal tubule epithelial cell' 'T cell'
 'kidney loop of Henle thick ascending limb epithelial cell'
 'Collecting duct intercalated cell' 'Kidney interstitial fibroblast'
 'Connecting tubule epithelial cell' 'Collecting duct principal cell'
 'Endothelial cell' 'B cell' 'Mononuclear phagocyte' 'Podocyte'
 'Parietal epithelial cell' 'Mast cell' 'Dendritic cell' 'Plasma cell'
 'Monocyte' 'Loop of Henle epithelial cell ' 'Renal interstitial pericyte'
 'Neutrophil' 'Papillary tips cell' 'neural cell']
23


In [124]:
cell_types_list=list(adata_concat_scvi.obs['general_cl_label'].unique())
marker_genes=[        "CCL21",
    "NRXN1",
    "SLC9A3",
    "SPP1",
    "SERPINE2",
    "IL18",
    "NBAS"
    "SLC9A3",
    "TIMP2",
    "IGFBP7",
    "SUGP1",
    "APOA1",
    "MMP9",
    "ITGAM",
    "TFF3",
    "B2M",
    "S100A6",
    "IL10",
    "IL6",
    "CXCL8",
    "LCN2",
    "HAVCR1",
    "CST3",
    "CCL2",
    "FABP1"]
# adata_concat_scvi.var_names=adata_concat_scvi.var['gene_name']

result = mean_expression_compare_to_normal(
    adata_concat_scvi,
    celltype_key="general_cl_label",
    as_id='as_id',
    disease_key="disease",
    selected_celltypes=cell_types_list,
    disease_list=['chronic kidney disease', 'acute kidney failure'],
    selected_genes=marker_genes,
    save_path="disease_vs_normal_mean_expression_long.csv"
)

print(result)

✅ Saved to disease_vs_normal_mean_expression_long.csv
          gene                 disease                        cell_type  \
0        CCL21  chronic kidney disease  Proximal tubule epithelial cell   
1        NRXN1  chronic kidney disease  Proximal tubule epithelial cell   
2       SLC9A3  chronic kidney disease  Proximal tubule epithelial cell   
3         SPP1  chronic kidney disease  Proximal tubule epithelial cell   
4     SERPINE2  chronic kidney disease  Proximal tubule epithelial cell   
...        ...                     ...                              ...   
1145      LCN2    acute kidney failure              Papillary tips cell   
1146    HAVCR1    acute kidney failure              Papillary tips cell   
1147      CST3    acute kidney failure              Papillary tips cell   
1148      CCL2    acute kidney failure              Papillary tips cell   
1149     FABP1    acute kidney failure              Papillary tips cell   

      mean_expression_disease  mean_expressio

In [ ]:
import scanpy as sc
import pandas as pd

def diffexp_compare_to_normal(adata, celltype_key, disease_key,
                              disease_list, selected_celltypes=None, selected_genes=None,
                              save_path=None):
    results = []

    for disease in disease_list:
        celltypes = selected_celltypes if selected_celltypes is not None else adata.obs[celltype_key].unique()

        for celltype in celltypes:
            mask = (adata.obs[celltype_key] == celltype) & adata.obs[disease_key].isin([disease, "normal"])
            adata_sub = adata[mask, :]

            if adata_sub.n_obs < 2:
                continue

            if selected_genes is not None:
                adata_sub = adata_sub[:, selected_genes]

            adata_sub.obs[disease_key] = adata_sub.obs[disease_key].astype(str)

            if "normal" not in adata_sub.obs[disease_key].unique():
                print(f"⚠️ Skipping {celltype} for {disease} (no 'normal' cells found)")
                continue

            if disease not in adata_sub.obs[disease_key].unique():
                print(f"⚠️ Skipping {celltype} for {disease} (no {disease} cells found)")
                continue

            sc.tl.rank_genes_groups(
                adata_sub,
                groupby=disease_key,
                groups=[disease],
                reference="normal",
                method="t-test",
                use_raw=False
            )

            res = adata_sub.uns["rank_genes_groups"]
            n_genes = len(res["names"][disease])

            for i in range(n_genes):
                results.append({
                    "gene": res["names"][disease][i],
                    "disease": disease,
                    "cell_type": celltype,
                    "log2FC": res["logfoldchanges"][disease][i],
                    "pval": res["pvals"][disease][i],
                    "pval_adj": res["pvals_adj"][disease][i]
                })

    df_out = pd.DataFrame(results)

    if save_path:
        df_out.to_csv(save_path, index=False)
        print(f"✅ Differential expression results saved to {save_path}")

    return df_out


In [92]:
len(list_cl_id)

23

In [107]:
df_ttest = diffexp_compare_to_normal(
    adata_concat_scvi,
    celltype_key="general_cl_label",
    disease_key="disease",
    disease_list=["acute kidney failure", "chronic kidney disease"],
    selected_celltypes=cell_types_list,
    selected_genes=marker_genes,
    save_path="disease_vs_normal_ttest.csv"
)

print(df_ttest.head())

/tmp/ipykernel_1229045/2401461207.py:33: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_sub.obs[disease_key] = adata_sub.obs[disease_key].astype(str)
/home/wruopeng/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_1229045/2401461207.py:33: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_sub.obs[disease_key] = adata_sub.obs[disease_key].astype(str)
/home/wruopeng/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_1229045/2401461207.py:33: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad

✅ Differential expression results saved to disease_vs_normal_ttest.csv
     gene               disease  \
0    SPP1  acute kidney failure   
1     B2M  acute kidney failure   
2  IGFBP7  acute kidney failure   
3    CST3  acute kidney failure   
4   TIMP2  acute kidney failure   

                                           cell_type    log2FC          pval  \
0  Kidney interstitial alternatively activated ma...  7.054031  8.593315e-14   
1  Kidney interstitial alternatively activated ma...  6.007898  3.436574e-12   
2  Kidney interstitial alternatively activated ma...  3.012910  2.164048e-09   
3  Kidney interstitial alternatively activated ma...  2.416271  2.572660e-07   
4  Kidney interstitial alternatively activated ma...  1.485941  1.396936e-05   

       pval_adj  
0  2.148329e-12  
1  4.295717e-11  
2  1.352530e-08  
3  1.286330e-06  
4  5.820568e-05  


In [127]:
df_merged = pd.merge(
    result,
    df_ttest,
    on=["gene", "disease", "cell_type"],
    how="left"   # keep all rows from df_means, add t-test stats if available
)

df_merged.to_csv("merged_expression_with_ttest.csv", index=False)

In [129]:
import mygene

import pandas as pd
import mygene



mg = mygene.MyGeneInfo()

genes = df_merged["gene"].unique().tolist()

# Batch query mygene
results = mg.querymany(
    genes,
    scopes="symbol",
    fields="ensembl.gene",
    species="human"
)

res_df = pd.DataFrame(results)[["query", "ensembl"]]

def extract_ensembl(x):
    if isinstance(x, list):
        return x[0].get("gene")
    elif isinstance(x, dict):
        return x.get("gene")
    else:
        return None

res_df["ensembl_id"] = res_df["ensembl"].apply(extract_ensembl)
res_df = res_df[["query", "ensembl_id"]].rename(columns={"query": "gene"})

df_merged = df_merged.merge(res_df, on="gene", how="left")

df_merged.to_csv("expression_with_ensembl.csv", index=False)

print("✅ Added Ensembl IDs. Saved as expression_with_ensembl.csv")
print(df_merged.head())


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


✅ Added Ensembl IDs. Saved as expression_with_ensembl.csv
     gene                 disease                        cell_type  \
0   CCL21  chronic kidney disease  Proximal tubule epithelial cell   
1   NRXN1  chronic kidney disease  Proximal tubule epithelial cell   
2  SLC9A3  chronic kidney disease  Proximal tubule epithelial cell   
3  SLC9A3  chronic kidney disease  Proximal tubule epithelial cell   
4    SPP1  chronic kidney disease  Proximal tubule epithelial cell   

   mean_expression_disease  mean_expression_normal  disease_sample_size  \
0                 0.018547                0.022178                34225   
1                 0.062998                0.024413                34225   
2                 0.643530                0.533689                34225   
3                 0.643530                0.533689                34225   
4                 5.240233                7.201705                34225   

   normal_sample_size  disease_female_sample_size  disease_male_sample

In [130]:
import requests

def get_cl_id(cell_type_name):
    url = "https://www.ebi.ac.uk/ols/api/search"
    params = {
        "q": cell_type_name,
        "ontology": "cl",     
        "exact": "true"        
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['response']['numFound'] > 0:
            hit = data['response']['docs'][0]
            return hit.get("obo_id")   # return just the CL ID
        else:
            return None
    else:
        response.raise_for_status()

celltype_to_cl = {}

for cell_type in cell_types_list:
    cl_id = get_cl_id(cell_type)
    celltype_to_cl[cell_type] = cl_id if cl_id is not None else "NA"

print(celltype_to_cl)



{'Kidney interstitial alternatively activated macrophage': 'CL:1000695', 'Distal convoluted tubule epithelial cell': 'CL:0002305', 'Proximal tubule epithelial cell': 'CL:0002306', 'T cell': 'CL:0000084', 'kidney loop of Henle thick ascending limb epithelial cell': 'CL:1001106', 'Collecting duct intercalated cell': 'CL:4030005', 'Kidney interstitial fibroblast': 'CL:1000692', 'Connecting tubule epithelial cell': 'CL:1000768', 'Collecting duct principal cell': 'CL:1000714', 'Endothelial cell': 'CL:0000115', 'B cell': 'CL:0000236', 'Mononuclear phagocyte': 'CL:0000113', 'Podocyte': 'CL:0000653', 'Parietal epithelial cell': 'CL:1000452', 'Mast cell': 'CL:0000097', 'Dendritic cell': 'CL:0000451', 'Plasma cell': 'CL:0000786', 'Monocyte': 'CL:0000576', 'Loop of Henle epithelial cell ': 'CL:1001016', 'Renal interstitial pericyte': 'CL:1001318', 'Neutrophil': 'CL:0000775', 'Papillary tips cell': 'CL:1000597', 'neural cell': 'CL:0002319'}


In [131]:
disease_mapping={"acute kidney failure": "HP_0001919","chronic kidney disease":"HP_0012622", "normal":"unknow"}

df_merged = df_merged.rename(columns={"mean_expression": "mean_expression_disease"})
df_merged["cl_id"] = df_merged["cell_type"].map(celltype_to_cl)
df_merged["disease_id"] = df_merged["disease"].map(disease_mapping)
df_merged["ratio_of_disease_over_normal_for_mean"] = (
    df_merged["mean_expression_disease"] / df_merged["mean_expression_normal"]
)

final_columns = [
    "disease",
    "disease_id",
    "cell_type",
    "cl_id",
    "gene",
    "ensembl_id",
    "mean_expression_disease",
    "disease_sample_size",
    "mean_expression_normal",
    "normal_sample_size",
    "disease_female_sample_size",
    "disease_male_sample_size",
    "normal_female_sample_size",
    "normal_male_sample_size",
    "disease_num_datasets",
    "normal_num_datasets",
    "disease_datasets",
    "normal_datasets",
    "ratio_of_disease_over_normal_for_mean",
    "log2FC",
    "pval_adj"
]

df_final = df_merged[final_columns]

df_final.to_csv("expression_reordered_final.csv", index=False)

In [160]:
df_final = pd.merge(
    df_final,
    df_UBERON,
    on=["cell_type"],
    how="left"   # keep all rows from df_means, add t-test stats if available
)
df_final.to_csv("expression_reordered_final_U.csv", index=False)